<a href="https://colab.research.google.com/github/Mikeblanchard/Covid_Project/blob/mike/Impute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import pearsonr
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
import scipy.sparse as sp

In [ ]:
dfd = pd.read_csv("Deaths_csv_cleaned.csv")
dfd

,Unnamed: 0,iso_code,continent,Country,date,total_cases,new_cases,total_deaths,total_cases_per_million,total_deaths_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,443,AFG,Asia,Afghanistan,2021-05-12,62718.0,315.0,2713.0,1611.114,69.692,NaN,3.892834e+07,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.500,64.83,0.511
1,898,OWID_AFR,NaN,Africa,2021-05-12,4656979.0,8863.0,125363.0,3473.807,93.513,NaN,1.340598e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1341,ALB,Europe,Albania,2021-05-12,131845.0,42.0,2423.0,45814.511,841.963,NaN,2.877800e+06,104.871,38.0,13.188,8.643,11803.431,1.1,304.195,10.08,7.100,51.200,NaN,2.890,78.57,0.795
3,1784,DZA,Africa,Algeria,2021-05-12,124682.0,199.0,3350.0,2843.308,76.395,NaN,4.385104e+07,17.348,29.1,6.211,3.857,13913.839,0.5,278.364,6.73,0.700,30.400,83.741,1.900,76.88,0.748
4,2221,AND,Europe,Andorra,2021-05-12,13470.0,23.0,127.0,174335.081,1643.694,NaN,7.726500e+04,163.755,NaN,NaN,NaN,NaN,NaN,109.135,7.97,29.000,37.800,NaN,NaN,83.73,0.868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,86224,WLF,Oceania,Wallis and Futuna,2021-05-11,NaN,NaN,NaN,NaN,NaN,NaN,1.124600e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.94,NaN
219,86701,OWID_WRL,NaN,World,2021-05-12,160450504.0,759730.0,3331258.0,20584.304,427.369,NaN,7.794799e+09,58.045,30.9,8.696,5.355,15469.207,10.0,233.070,8.51,6.434,34.635,60.130,2.705,72.58,0.737
220,87099,YEM,Asia,Yemen,2021-05-12,6498.0,6.0,1277.0,217.864,42.815,NaN,2.982597e+07,53.508,20.3,2.922,1.583,1479.147,18.8,495.003,5.35,7.600,29.200,49.542,0.700,66.12,0.470
221,87520,ZMB,Africa,Zambia,2021-05-12,92211.0,59.0,1259.0,5015.841,68.484,NaN,1.838396e+07,22.995,17.7,2.480,1.542,3689.251,57.5,234.499,3.94,3.100,24.700,13.938,2.000,63.89,0.584


In [ ]:
df_dropped = dfd.drop(["Unnamed: 0", "iso_code", "continent", "Country", "date", "stringency_index", "handwashing_facilities", "extreme_poverty", "male_smokers", "female_smokers"], 1)
df_dropped

,total_cases,new_cases,total_deaths,total_cases_per_million,total_deaths_per_million,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,hospital_beds_per_thousand,life_expectancy,human_development_index
0,62718.0,315.0,2713.0,1611.114,69.692,3.892834e+07,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,0.500,64.83,0.511
1,4656979.0,8863.0,125363.0,3473.807,93.513,1.340598e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,131845.0,42.0,2423.0,45814.511,841.963,2.877800e+06,104.871,38.0,13.188,8.643,11803.431,304.195,10.08,2.890,78.57,0.795
3,124682.0,199.0,3350.0,2843.308,76.395,4.385104e+07,17.348,29.1,6.211,3.857,13913.839,278.364,6.73,1.900,76.88,0.748
4,13470.0,23.0,127.0,174335.081,1643.694,7.726500e+04,163.755,NaN,NaN,NaN,NaN,109.135,7.97,NaN,83.73,0.868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,NaN,NaN,NaN,NaN,NaN,1.124600e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.94,NaN
219,160450504.0,759730.0,3331258.0,20584.304,427.369,7.794799e+09,58.045,30.9,8.696,5.355,15469.207,233.070,8.51,2.705,72.58,0.737
220,6498.0,6.0,1277.0,217.864,42.815,2.982597e+07,53.508,20.3,2.922,1.583,1479.147,495.003,5.35,0.700,66.12,0.470
221,92211.0,59.0,1259.0,5015.841,68.484,1.838396e+07,22.995,17.7,2.480,1.542,3689.251,234.499,3.94,2.000,63.89,0.584


In [ ]:
df_dropped.isnull().sum()

total_cases                   24
new_cases                     24
total_deaths                  31
total_cases_per_million       25
total_deaths_per_million      32
population                     2
population_density            20
median_age                    34
aged_65_older                 36
aged_70_older                 35
gdp_per_capita                32
cardiovasc_death_rate         35
diabetes_prevalence           25
hospital_beds_per_thousand    54
life_expectancy               12
human_development_index       36
dtype: int64

In [ ]:
impute = IterativeImputer(max_iter=20, verbose=20)
df_array = impute.fit_transform(df_dropped)
df_array

[IterativeImputer] Completing matrix with shape (223, 16)
[IterativeImputer] Ending imputation round 1/20, elapsed time 0.04
[IterativeImputer] Change: 111700286.49069183, scaled tolerance: 7794798.729 
[IterativeImputer] Ending imputation round 2/20, elapsed time 0.07
[IterativeImputer] Change: 1786103.8812536022, scaled tolerance: 7794798.729 
[IterativeImputer] Early stopping criterion reached.


array([[6.27180000e+04, 3.15000000e+02, 2.71300000e+03, ...,
        5.00000000e-01, 6.48300000e+01, 5.11000000e-01],
       [4.65697900e+06, 8.86300000e+03, 1.25363000e+05, ...,
        3.81031107e+00, 7.08021063e+01, 6.62251301e-01],
       [1.31845000e+05, 4.20000000e+01, 2.42300000e+03, ...,
        2.89000000e+00, 7.85700000e+01, 7.95000000e-01],
       ...,
       [6.49800000e+03, 6.00000000e+00, 1.27700000e+03, ...,
        7.00000000e-01, 6.61200000e+01, 4.70000000e-01],
       [9.22110000e+04, 5.90000000e+01, 1.25900000e+03, ...,
        2.00000000e+00, 6.38900000e+01, 5.84000000e-01],
       [3.84660000e+04, 1.80000000e+01, 1.57900000e+03, ...,
        1.70000000e+00, 6.14900000e+01, 5.71000000e-01]])

In [ ]:
df = pd.DataFrame(df_array, index=df_dropped.index, columns=df_dropped.columns)
df.head(20)

In [ ]:
df = df.join(dfd["Country"])
df

,total_cases,new_cases,total_deaths,total_cases_per_million,total_deaths_per_million,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,hospital_beds_per_thousand,life_expectancy,human_development_index,Country
0,6.271800e+04,315.00000,2.713000e+03,1611.114000,69.692000,3.892834e+07,54.422000,18.600000,2.581000,1.337000,1803.987000,597.029000,9.590000,0.500000,64.830000,0.511000,Afghanistan
1,4.656979e+06,8863.00000,1.253630e+05,3473.807000,93.513000,1.340598e+09,111.790733,31.790057,8.689491,5.265657,9850.278742,327.431344,4.439204,3.810311,70.802106,0.662251,Africa
2,1.318450e+05,42.00000,2.423000e+03,45814.511000,841.963000,2.877800e+06,104.871000,38.000000,13.188000,8.643000,11803.431000,304.195000,10.080000,2.890000,78.570000,0.795000,Albania
3,1.246820e+05,199.00000,3.350000e+03,2843.308000,76.395000,4.385104e+07,17.348000,29.100000,6.211000,3.857000,13913.839000,278.364000,6.730000,1.900000,76.880000,0.748000,Algeria
4,1.347000e+04,23.00000,1.270000e+02,174335.081000,1643.694000,7.726500e+04,163.755000,37.431900,9.212445,6.139287,58750.184450,109.135000,7.970000,2.777435,83.730000,0.868000,Andorra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,2.344393e+06,2910.67445,5.734865e+04,34677.841632,639.153173,1.124600e+04,466.849047,33.126534,8.913775,5.587998,20633.070072,250.224178,10.616761,3.122759,79.940000,0.810548,Wallis and Futuna
219,1.604505e+08,759730.00000,3.331258e+06,20584.304000,427.369000,7.794799e+09,58.045000,30.900000,8.696000,5.355000,15469.207000,233.070000,8.510000,2.705000,72.580000,0.737000,World
220,6.498000e+03,6.00000,1.277000e+03,217.864000,42.815000,2.982597e+07,53.508000,20.300000,2.922000,1.583000,1479.147000,495.003000,5.350000,0.700000,66.120000,0.470000,Yemen
221,9.221100e+04,59.00000,1.259000e+03,5015.841000,68.484000,1.838396e+07,22.995000,17.700000,2.480000,1.542000,3689.251000,234.499000,3.940000,2.000000,63.890000,0.584000,Zambia


In [ ]:
# co = df["Country"]
# df.drop(labels=["Country"], axis=1, inplace=True)
# df.insert(0, "Country", co)


In [ ]:
colu = list(df)
colu.insert(0, colu.pop(colu.index("Country")))

colu

['Country',
 'total_cases',
 'new_cases',
 'total_deaths',
 'total_cases_per_million',
 'total_deaths_per_million',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'cardiovasc_death_rate',
 'diabetes_prevalence',
 'hospital_beds_per_thousand',
 'life_expectancy',
 'human_development_index']

In [ ]:
df = df.loc[:, colu]
df.head(20)

,Country,total_cases,new_cases,total_deaths,total_cases_per_million,total_deaths_per_million,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,hospital_beds_per_thousand,life_expectancy,human_development_index
0,Afghanistan,6.271800e+04,315.000000,2713.000000,1611.114000,69.692000,3.892834e+07,54.422000,18.600000,2.581000,1.337000,1803.987000,597.029000,9.590000,0.500000,64.830000,0.511000
1,Africa,4.656979e+06,8863.000000,125363.000000,3473.807000,93.513000,1.340598e+09,111.790733,31.790057,8.689491,5.265657,9850.278742,327.431344,4.439204,3.810311,70.802106,0.662251
2,Albania,1.318450e+05,42.000000,2423.000000,45814.511000,841.963000,2.877800e+06,104.871000,38.000000,13.188000,8.643000,11803.431000,304.195000,10.080000,2.890000,78.570000,0.795000
3,Algeria,1.246820e+05,199.000000,3350.000000,2843.308000,76.395000,4.385104e+07,17.348000,29.100000,6.211000,3.857000,13913.839000,278.364000,6.730000,1.900000,76.880000,0.748000
4,Andorra,1.347000e+04,23.000000,127.000000,174335.081000,1643.694000,7.726500e+04,163.755000,37.431900,9.212445,6.139287,58750.184450,109.135000,7.970000,2.777435,83.730000,0.868000
5,Angola,2.940500e+04,259.000000,645.000000,894.686000,19.625000,3.286627e+07,23.890000,16.800000,2.405000,1.362000,5819.495000,276.045000,3.940000,0.839023,61.150000,0.581000
6,Anguilla,2.344408e+06,2911.040910,57348.363737,34682.839958,639.229936,1.500200e+04,466.881992,33.891894,8.986428,5.610260,20634.893382,250.218717,11.155920,3.188409,81.880000,0.832304
7,Antigua and Barbuda,1.240000e+03,2.000000,32.000000,12662.364000,326.771000,9.792800e+04,231.845000,32.100000,6.933000,4.631000,21490.943000,191.511000,13.170000,3.800000,77.020000,0.778000
8,Argentina,3.215572e+06,24475.000000,68807.000000,71147.621000,1522.421000,4.519578e+07,16.177000,31.900000,11.198000,7.441000,18933.907000,191.032000,5.500000,5.000000,76.670000,0.845000
9,Armenia,2.199500e+05,354.000000,4272.000000,74226.335000,1441.668000,2.963234e+06,102.931000,35.700000,11.232000,7.571000,8787.580000,341.010000,7.110000,4.200000,75.090000,0.776000


In [ ]:
df.to_csv("Impute_Deaths.csv")